In [9]:
import librosa
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

In [4]:
def load_audio(audio_file_path):
    audio, sr = librosa.load(audio_file_path, sr=44100)
    return audio, sr

def load_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

In [12]:
def extract_features(audio, sr):
    """
    Extracts multiple audio features (MFCC, Chroma STFT, Chroma CQT, Chroma CENS) from the audio signal.
    Returns a 3D vector (time, frequency, feature).
    """
    # Feature 1: MFCC
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)

    # Feature 2: Chroma STFT
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    chroma_stft_mean = np.mean(chroma_stft, axis=1)

    # Feature 3: Chroma CQT with reduced bins
    chroma_cqt = librosa.feature.chroma_cqt(y=audio, sr=sr, bins_per_octave=24, n_octaves=7)
    chroma_cqt_mean = np.mean(chroma_cqt, axis=1)

    # Feature 4: Chroma CENS
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr)
    chroma_cens_mean = np.mean(chroma_cens, axis=1)

    # Combine all features into a single vector
    return np.hstack([mfcc_mean, chroma_stft_mean, chroma_cqt_mean, chroma_cens_mean])


In [28]:
audio_file_path = "7061-6-0-0.wav"
audio,sr=load_audio(audio_file_path)
features = extract_features(audio,sr)


/home/aditya/ADITYA/Projects/Denoise/myenv/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=776
  warnings.warn(


In [ ]:
def scale(features):
    scaled_features = []
    minimum = np.min(features) 
    maximum = np.max(features) 

    for value in features:
        scaled_value = (value - minimum) / (maximum - minimum)  
        scaled_features.append(scaled_value)  
    scaled_features = np.array(scaled_features)

    return scaled_features

scaled_features = scale(features)

[0.         1.         0.5628778  0.8164663  0.70170605 0.8815811
 0.7686461  0.84498256 0.7965568  0.8806914  0.8470614  0.8955136
 0.8857837  0.81819165 0.8184743  0.81763494 0.8177113  0.8182802
 0.81908554 0.81912124 0.8184706  0.817888   0.81751215 0.8173311
 0.8174683  0.8183508  0.81874883 0.81889325 0.8188635  0.8188479
 0.8188083  0.8188492  0.81890476 0.8191638  0.81810915 0.8178538
 0.8178167  0.8168783  0.81690925 0.81710756 0.8169514  0.81696516
 0.8170699  0.81694156 0.8172797  0.8169106  0.8167266  0.81659716
 0.8167198 ]
